
# Environment Setup & Configuration


In [13]:
import bigframes.pandas as bpd
import bigframes.bigquery as bbq
from bigframes.ml.llm import TextEmbeddingGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 🔧 Project Configuration - CHANGE THESE TO YOUR VALUES
PROJECT_ID = "bigquery-hackathon-471715"  # Your project ID
DATASET_ID = "thelook_ecommerce"          # Your dataset
LOCATION = "US"                           # Your location

# Configure BigFrames
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = LOCATION
bpd.options.display.max_columns = None

print("Environment configured successfully!")
print(f"Project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")

# Quick data quality check
try:
    test_query = f"""
    SELECT COUNT(*) as product_count,
           COUNT(embedding) as embeddings_count,
           COUNT(DISTINCT category) as categories
    FROM `{PROJECT_ID}.{DATASET_ID}.product_embeddings_clean`
    """
    quality_check = bpd.read_gbq(test_query).to_pandas().iloc[0]
    print(f"Data Status: {quality_check['product_count']} products, {quality_check['embeddings_count']} embeddings, {quality_check['categories']} categories")
except Exception as e:
    print(f"Warning: Could not connect to data. Error: {str(e)}")

Environment configured successfully!
Project: bigquery-hackathon-471715
Dataset: thelook_ecommerce
Data Status: 29094 products, 29094 embeddings, 26 categories


# Smart Product Explorer & Search

In [14]:
class SmartProductExplorer:
    """Enhanced product exploration with intelligent filtering"""

    def __init__(self, project_id, dataset_id):
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.products_df = None

    def load_products_with_insights(self, limit=30000):
        """Load products with additional insights"""
        query = f"""
        WITH product_insights AS (
            SELECT
                p.id, p.name, p.category, p.brand, p.retail_price,
                ARRAY_LENGTH(p.embedding) as embedding_dim,
                CASE
                    WHEN p.retail_price < 20 THEN 'Budget (Under $20)'
                    WHEN p.retail_price < 50 THEN 'Mid-Range ($20-$50)'
                    WHEN p.retail_price < 100 THEN 'Premium ($50-$100)'
                    ELSE 'Luxury ($100+)'
                END as price_tier,
                COUNT(*) OVER (PARTITION BY p.brand) as brand_product_count,
                COUNT(*) OVER (PARTITION BY p.category) as category_size
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
        )
        SELECT *,
            CASE
                WHEN brand_product_count > 100 THEN 'Major Brand'
                WHEN brand_product_count > 20 THEN 'Popular Brand'
                ELSE 'Boutique Brand'
            END as brand_tier
        FROM product_insights
        ORDER BY retail_price DESC
        LIMIT {limit}
        """

        self.products_df = bpd.read_gbq(query)
        print(f"Loaded {len(self.products_df)} products with insights")
        return self.products_df

    def smart_search(self, search_term, max_results=10):
        """Intelligent product search with multiple criteria"""
        if self.products_df is None:
            self.load_products_with_insights()

        # Convert to pandas for complex filtering
        df = self.products_df.to_pandas()

        # Multi-criteria search
        search_lower = search_term.lower()
        matches = df[
            df['name'].str.lower().str.contains(search_lower, na=False) |
            df['category'].str.lower().str.contains(search_lower, na=False) |
            df['brand'].str.lower().str.contains(search_lower, na=False)
        ]

        print(f"Found {len(matches)} products matching '{search_term}'")
        return matches.head(max_results)

    def get_category_insights(self):
        """Get comprehensive category insights"""
        df = self.products_df.to_pandas() if self.products_df is not None else None
        if df is None:
            self.load_products_with_insights()
            df = self.products_df.to_pandas()

        category_stats = df.groupby('category').agg({
            'id': 'count',
            'retail_price': ['mean', 'min', 'max'],
            'brand': 'nunique'
        }).round(2)

        category_stats.columns = ['product_count', 'avg_price', 'min_price', 'max_price', 'unique_brands']
        return category_stats.sort_values('product_count', ascending=False)

# Advanced Recommendation Engine

In [15]:
class AdvancedRecommendationEngine:
    """State-of-the-art recommendation engine with multiple algorithms"""

    def __init__(self, project_id, dataset_id):
        self.project_id = project_id
        self.dataset_id = dataset_id
        self.recommendation_cache = {}

    def get_product_info(self, product_id):
        """Get detailed product information"""
        query = f"""
        SELECT id, name, category, brand, retail_price
        FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
        WHERE id = {product_id}
        """
        result = bpd.read_gbq(query)
        return result.to_pandas().iloc[0] if result.shape[0] > 0 else None

    def semantic_recommendations(self, product_id, top_k=5):
        """Pure semantic similarity recommendations"""
        query = f"""
        WITH target AS (
            SELECT embedding, name as target_name, category, brand, retail_price
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
            WHERE id = {product_id}
        )
        SELECT
            p.id, p.name, p.category, p.brand, p.retail_price,
            t.target_name,
            ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 4) AS semantic_similarity,
            'Pure Semantic' as algorithm_type
        FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
        CROSS JOIN target t
        WHERE p.id != {product_id}
        ORDER BY semantic_similarity DESC
        LIMIT {top_k}
        """
        return bpd.read_gbq(query).to_pandas()

    def hybrid_recommendations(self, product_id, top_k=5, semantic_weight=0.6, price_weight=0.25, brand_weight=0.1, category_weight=0.05):
        """Advanced hybrid recommendations with customizable weights"""
        query = f"""
        WITH target AS (
            SELECT embedding, name as target_name, category, brand, retail_price
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
            WHERE id = {product_id}
        ),
        hybrid_scores AS (
            SELECT
                p.id, p.name, p.category, p.brand, p.retail_price,
                t.target_name,
                -- Individual scores
                ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 4) AS semantic_sim,
                ROUND(1 - (ABS(p.retail_price - t.retail_price) / GREATEST(p.retail_price, t.retail_price)), 4) AS price_sim,
                CASE WHEN p.brand = t.brand THEN 1.0 ELSE 0.0 END AS brand_match,
                CASE WHEN p.category = t.category THEN 1.0 ELSE 0.0 END AS category_match,

                -- Hybrid score
                ROUND(
                    (1 - COSINE_DISTANCE(t.embedding, p.embedding)) * {semantic_weight} +
                    (1 - ABS(p.retail_price - t.retail_price) / GREATEST(p.retail_price, t.retail_price)) * {price_weight} +
                    CASE WHEN p.brand = t.brand THEN {brand_weight} ELSE 0 END +
                    CASE WHEN p.category = t.category THEN {category_weight} ELSE 0 END, 4
                ) AS hybrid_score,

                -- Explanation
                CASE
                    WHEN p.brand = t.brand AND p.category = t.category THEN 'Perfect Match (Same Brand & Category)'
                    WHEN p.brand = t.brand THEN 'Brand Loyalty Match'
                    WHEN p.category = t.category THEN 'Category Match'
                    WHEN ABS(p.retail_price - t.retail_price) / t.retail_price < 0.15 THEN 'Price Point Match'
                    ELSE 'AI Semantic Discovery'
                END as match_explanation

            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
            CROSS JOIN target t
            WHERE p.id != {product_id}
        )
        SELECT *, 'Hybrid AI' as algorithm_type
        FROM hybrid_scores
        ORDER BY hybrid_score DESC
        LIMIT {top_k}
        """
        return bpd.read_gbq(query).to_pandas()

    def business_smart_recommendations(self, product_id, top_k=5):
        """Business-optimized recommendations considering profit margins"""
        query = f"""
        WITH target AS (
            SELECT p.embedding, p.name as target_name, p.category, p.brand, p.retail_price,
                   orig.cost
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
            LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` orig ON p.id = orig.id
            WHERE p.id = {product_id}
        ),
        business_recs AS (
            SELECT
                p.id, p.name, p.category, p.brand, p.retail_price,
                orig.cost,
                CASE WHEN orig.cost IS NOT NULL THEN
                    ROUND((orig.retail_price - orig.cost) / orig.retail_price * 100, 1)
                    ELSE NULL END as margin_pct,
                ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 4) AS semantic_similarity,

                -- Business score (higher margin products get bonus)
                ROUND(
                    (1 - COSINE_DISTANCE(t.embedding, p.embedding)) * 0.7 +
                    CASE WHEN orig.cost IS NOT NULL AND (orig.retail_price - orig.cost) / orig.retail_price > 0.4
                         THEN 0.3 ELSE 0.1 END, 4
                ) AS business_score,

                CASE
                    WHEN orig.cost IS NOT NULL AND (orig.retail_price - orig.cost) / orig.retail_price > 0.5
                        THEN 'High Profit Recommendation'
                    WHEN orig.cost IS NOT NULL AND (orig.retail_price - orig.cost) / orig.retail_price > 0.3
                        THEN 'Good Profit Recommendation'
                    ELSE 'Standard Recommendation'
                END as business_tier

            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
            LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` orig ON p.id = orig.id
            CROSS JOIN target t
            WHERE p.id != {product_id}
        )
        SELECT *, 'Business Smart' as algorithm_type
        FROM business_recs
        ORDER BY business_score DESC
        LIMIT {top_k}
        """
        return bpd.read_gbq(query).to_pandas()

    def comprehensive_recommendation_comparison(self, product_id):
        """Compare all recommendation algorithms side by side"""
        product_info = self.get_product_info(product_id)
        if product_info is None:
            return None

        print(f"COMPREHENSIVE RECOMMENDATIONS FOR:")
        print(f"   Product: {product_info['name']}")
        print(f"   Category: {product_info['category']} | Brand: {product_info['brand']} | Price: ${product_info['retail_price']:.2f}")
        print("=" * 100)

        # Get recommendations from all algorithms
        algorithms = {
            'Semantic': self.semantic_recommendations(product_id, 3),
            'Hybrid': self.hybrid_recommendations(product_id, 3),
            'Business': self.business_smart_recommendations(product_id, 3)
        }

        for algo_name, recs in algorithms.items():
            print(f"\n{algo_name} RECOMMENDATIONS:")
            print("-" * 50)
            for i, (_, row) in enumerate(recs.iterrows(), 1):
                print(f"  {i}. {row['name'][:50]}...")
                print(f"     Category: {row['category']} | Brand: {row['brand']} | Price: ${row['retail_price']:.2f}")

                # Algorithm-specific scores
                if 'semantic_similarity' in row:
                    score = row.get('hybrid_score', row.get('business_score', row['semantic_similarity']))
                    print(f"     Score: {score:.3f}", end="")
                    if 'match_explanation' in row:
                        print(f" | {row['match_explanation']}")
                    elif 'business_tier' in row:
                        print(f" | {row['business_tier']}")
                    else:
                        print(" | Semantic Similarity")

        return algorithms

# Interactive Visualization Dashboard

In [16]:
class InteractiveVisualizationDashboard:
    """Create beautiful, interactive visualizations for vector search insights"""

    def __init__(self, project_id, dataset_id):
        self.project_id = project_id
        self.dataset_id = dataset_id

    def create_similarity_heatmap(self, sample_products=None):
        """Create a similarity heatmap for a set of products"""
        if sample_products is None:
            # Get a diverse sample of products
            sample_query = f"""
            SELECT DISTINCT id, name, category
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
            ORDER BY RAND()
            LIMIT 8
            """
            sample_products = bpd.read_gbq(sample_query).to_pandas()['id'].tolist()

        # Get similarity matrix
        products_str = ','.join(map(str, sample_products[:8]))  # Limit to 8 for readability

        similarity_query = f"""
        WITH sample_products AS (
            SELECT id, name, embedding
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
            WHERE id IN ({products_str})
        )
        SELECT
            p1.id as product1_id,
            p1.name as product1_name,
            p2.id as product2_id,
            p2.name as product2_name,
            ROUND((1 - COSINE_DISTANCE(p1.embedding, p2.embedding)), 3) AS similarity
        FROM sample_products p1
        CROSS JOIN sample_products p2
        """

        similarity_data = bpd.read_gbq(similarity_query).to_pandas()

        # Create pivot table for heatmap
        pivot_table = similarity_data.pivot(index='product1_name', columns='product2_name', values='similarity')

        # Create interactive heatmap
        fig = px.imshow(
            pivot_table.values,
            x=[name[:20] + '...' if len(name) > 20 else name for name in pivot_table.columns],
            y=[name[:20] + '...' if len(name) > 20 else name for name in pivot_table.index],
            color_continuous_scale='RdYlBu_r',
            title='Product Similarity Heatmap (Cosine Similarity)',
            labels={'color': 'Similarity Score'}
        )

        fig.update_layout(
            width=800, height=600,
            title_x=0.5,
            xaxis_title="Products",
            yaxis_title="Products"
        )

        fig.show()
        return fig

    def create_category_performance_dashboard(self):
        """Comprehensive category performance visualization"""

        # Get category data
        category_query = f"""
        WITH category_metrics AS (
            SELECT
                p.category,
                COUNT(*) as product_count,
                AVG(p.retail_price) as avg_price,
                MIN(p.retail_price) as min_price,
                MAX(p.retail_price) as max_price,
                COUNT(DISTINCT p.brand) as unique_brands,
                AVG(orig.cost) as avg_cost,
                AVG(CASE WHEN orig.cost IS NOT NULL
                    THEN (orig.retail_price - orig.cost) / orig.retail_price
                    ELSE NULL END) as avg_margin
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
            LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` orig ON p.id = orig.id
            GROUP BY p.category
        )
        SELECT *,
            CASE
                WHEN avg_margin > 0.5 THEN 'High Profit'
                WHEN avg_margin > 0.3 THEN 'Good Profit'
                WHEN avg_margin > 0.15 THEN 'Average Profit'
                ELSE 'Low Profit'
            END as profit_tier
        FROM category_metrics
        ORDER BY product_count DESC
        """

        category_data = bpd.read_gbq(category_query).to_pandas()

        # Create subplot dashboard
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Products per Category',
                'Average Price by Category',
                'Brand Diversity',
                'Profit Margins'
            ),
            specs=[[{"type": "bar"}, {"type": "bar"}],
                   [{"type": "scatter"}, {"type": "bar"}]]
        )

        # Top 10 categories for readability
        top_categories = category_data.head(10)

        # Product count
        fig.add_trace(
            go.Bar(x=top_categories['category'], y=top_categories['product_count'],
                   name="Product Count", marker_color='lightblue'),
            row=1, col=1
        )

        # Average price
        fig.add_trace(
            go.Bar(x=top_categories['category'], y=top_categories['avg_price'],
                   name="Avg Price", marker_color='lightcoral'),
            row=1, col=2
        )

        # Brand diversity vs product count
        fig.add_trace(
            go.Scatter(x=top_categories['product_count'], y=top_categories['unique_brands'],
                      mode='markers+text', text=top_categories['category'],
                      textposition='top center', name="Brand Diversity",
                      marker=dict(size=10, color='green')),
            row=2, col=1
        )

        # Profit margins (where available)
        margin_data = top_categories[top_categories['avg_margin'].notna()]
        if not margin_data.empty:
            fig.add_trace(
                go.Bar(x=margin_data['category'], y=margin_data['avg_margin']*100,
                       name="Profit Margin %", marker_color='gold'),
                row=2, col=2
            )

        fig.update_layout(
            height=800,
            title_text="E-Commerce Category Performance Dashboard",
            title_x=0.5,
            showlegend=False
        )

        # Update subplot titles
        fig.update_xaxes(tickangle=45)

        fig.show()
        return fig

    def visualize_recommendation_quality(self, product_id, algorithms_results):
        """Visualize recommendation quality across different algorithms"""

        # Prepare data for visualization
        all_scores = []
        for algo_name, recs_df in algorithms_results.items():
            for _, row in recs_df.iterrows():
                score_col = 'hybrid_score' if 'hybrid_score' in row else \
                           'business_score' if 'business_score' in row else 'semantic_similarity'

                explanation = ""
                if 'match_explanation' in row:
                  explanation = row['match_explanation']
                elif 'business_tier' in row:
                  explanation = row['business_tier']

                all_scores.append({
                    'Algorithm': algo_name,
                    'Product': row['name'][:30] + '...' if len(row['name']) > 30 else row['name'],
                    'Score': row[score_col],
                    'Category': row['category'],
                    'Price': row['retail_price'],
                    'Explanation': explanation # Include explanation for hover info
                })

        scores_df = pd.DataFrame(all_scores)

        # Create comparison chart
        fig = px.bar(
            scores_df,
            x='Product',
            y='Score',
            color='Algorithm',
            title=f'Recommendation Quality Comparison for Product {product_id}',
            hover_data=['Category', 'Price', 'Explanation'] # Add Explanation to hover data
        )

        fig.update_layout(
            width=1000, height=500,
            title_x=0.5,
            xaxis_tickangle=45
        )

        fig.show()
        return fig

# Business Intelligence & Analytics Engine

In [17]:
class BusinessIntelligenceEngine:
    """Advanced business analytics using vector search insights"""

    def __init__(self, project_id, dataset_id):
        self.project_id = project_id
        self.dataset_id = dataset_id

    def analyze_cross_selling_opportunities(self):
        """Find the best cross-selling opportunities using semantic similarity"""
        query = f"""
        WITH cross_sell_analysis AS (
            SELECT
                p1.category as primary_category,
                p2.category as cross_sell_category,
                COUNT(*) as similar_product_pairs,
                AVG((1 - COSINE_DISTANCE(p1.embedding, p2.embedding))) as avg_similarity,
                AVG(p1.retail_price) as avg_primary_price,
                AVG(p2.retail_price) as avg_cross_sell_price,
                AVG(p1.retail_price + p2.retail_price) as avg_bundle_value
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            CROSS JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
            WHERE p1.category != p2.category
              AND (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) > 0.4
            GROUP BY p1.category, p2.category
            HAVING COUNT(*) >= 10  -- Ensure statistical significance
        )
        SELECT
            primary_category,
            cross_sell_category,
            similar_product_pairs,
            ROUND(avg_similarity, 3) as semantic_affinity,
            ROUND(avg_primary_price, 2) as avg_primary_price,
            ROUND(avg_cross_sell_price, 2) as avg_cross_sell_price,
            ROUND(avg_bundle_value, 2) as potential_bundle_value,
            CASE
                WHEN avg_similarity > 0.6 THEN 'Excellent Cross-Sell'
                WHEN avg_similarity > 0.5 THEN 'Good Cross-Sell'
                ELSE 'Moderate Cross-Sell'
            END as opportunity_rating
        FROM cross_sell_analysis
        ORDER BY avg_similarity DESC, similar_product_pairs DESC
        LIMIT 20
        """

        return bpd.read_gbq(query).to_pandas()

    def identify_pricing_opportunities(self):
        """Identify products that could be repriced based on similar product analysis"""
        query = f"""
        WITH similar_products_analysis AS (
            SELECT
                p1.id as product_id,
                p1.retail_price as current_price,
                p2.retail_price as similar_price,
                (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) as similarity_score
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
              ON p1.category = p2.category AND p1.id != p2.id
            WHERE (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) > 0.6
              AND ABS(p2.retail_price - p1.retail_price) / p1.retail_price < 0.5
        ),
        pricing_analysis AS (
            SELECT
                product_id,
                ANY_VALUE(current_price) as current_price,
                AVG(similar_price) as market_avg_price,
                COUNT(similar_price) as similar_products_count
            FROM similar_products_analysis
            GROUP BY product_id
            HAVING COUNT(similar_price) >= 3
        )
        SELECT
            p.id,
            p.name,
            p.category,
            p.brand,
            ROUND(pa.current_price, 2) as current_price,
            ROUND(pa.market_avg_price, 2) as market_avg_price,
            pa.similar_products_count,
            ROUND(pa.market_avg_price - pa.current_price, 2) as price_gap,
            ROUND((pa.market_avg_price - pa.current_price) / pa.current_price * 100, 1) as price_gap_percent,
            CASE
                WHEN pa.market_avg_price > pa.current_price * 1.15 THEN 'Under-priced (Increase Opportunity)'
                WHEN pa.market_avg_price < pa.current_price * 0.85 THEN 'Over-priced (Decrease Risk)'
                ELSE 'Well-priced'
            END as pricing_recommendation,
            CASE
                WHEN pa.similar_products_count >= 10 THEN 'High Confidence'
                WHEN pa.similar_products_count >= 5 THEN 'Medium Confidence'
                ELSE 'Low Confidence'
            END as confidence_level
        FROM pricing_analysis pa
        JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p ON pa.product_id = p.id
        ORDER BY ABS(price_gap_percent) DESC
        LIMIT 30
        """

        return bpd.read_gbq(query).to_pandas()

    def generate_product_clustering_insights(self):
        """Advanced product clustering based on vector embeddings"""
        query = f"""
        WITH product_similarities AS (
            SELECT
                p1.id as product_id,
                p1.name,
                p1.category,
                p1.brand,
                p1.retail_price,
                -- Find the most similar product that's NOT in the same category
                ARRAY_AGG(
                    STRUCT(
                        p2.id as similar_id,
                        p2.name as similar_name,
                        p2.category as similar_category,
                        (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) as similarity
                    )
                    ORDER BY (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) DESC
                    LIMIT 3
                )[OFFSET(0)] as most_similar_different_category
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            CROSS JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
            WHERE p1.id != p2.id
              AND p1.category != p2.category
              AND (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) > 0.3
            GROUP BY p1.id, p1.name, p1.category, p1.brand, p1.retail_price
        )
        SELECT
            product_id,
            name as product_name,
            category as product_category, -- Corrected alias here
            brand,
            retail_price,
            most_similar_different_category.similar_name as cross_category_match,
            most_similar_different_category.similar_category as match_category,
            ROUND(most_similar_different_category.similarity, 3) as cross_category_similarity,
            CASE
                WHEN most_similar_different_category.similarity > 0.7 THEN 'Strong Cross-Category Appeal'
                WHEN most_similar_different_category.similarity > 0.5 THEN 'Moderate Cross-Category Appeal'
                ELSE 'Category-Specific'
            END as market_positioning
        FROM product_similarities
        WHERE most_similar_different_category IS NOT NULL
        ORDER BY most_similar_different_category.similarity DESC
        LIMIT 25
        """

        return bpd.read_gbq(query).to_pandas()

    def analyze_brand_positioning(self):
        """Analyze how brands position themselves in the semantic space"""
        query = f"""
        WITH brand_analysis AS (
            SELECT
                p1.brand,
                p2.brand as competitor_brand,
                COUNT(*) as similar_product_pairs,
                AVG((1 - COSINE_DISTANCE(p1.embedding, p2.embedding))) as avg_similarity,
                AVG(p1.retail_price) as avg_brand_price,
                AVG(p2.retail_price) as avg_competitor_price
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            CROSS JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
            WHERE p1.brand != p2.brand
              AND (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) > 0.5
            GROUP BY p1.brand, p2.brand
            HAVING COUNT(*) >= 5
        ),
        brand_metrics AS (
            SELECT
                brand,
                COUNT(DISTINCT competitor_brand) as competing_brands,
                AVG(avg_similarity) as overall_market_similarity,
                AVG(avg_brand_price) as brand_price_point,
                AVG(similar_product_pairs) as avg_competing_products
            FROM brand_analysis
            GROUP BY brand
        )
        SELECT
            brand,
            competing_brands,
            ROUND(overall_market_similarity, 3) as market_similarity_score,
            ROUND(brand_price_point, 2) as avg_price_point,
            ROUND(avg_competing_products, 1) as avg_competing_products,
            CASE
                WHEN overall_market_similarity > 0.65 THEN 'Highly Competitive Space'
                WHEN overall_market_similarity > 0.55 THEN 'Moderate Competition'
                ELSE 'Niche Positioning'
            END as competitive_landscape,
            CASE
                WHEN brand_price_point > 75 THEN 'Premium Brand'
                WHEN brand_price_point > 35 THEN 'Mid-Market Brand'
                ELSE 'Value Brand'
            END as price_positioning
        FROM brand_metrics
        WHERE competing_brands >= 3
        ORDER BY overall_market_similarity DESC, competing_brands DESC
        LIMIT 20
        """

        return bpd.read_gbq(query).to_pandas()

    def generate_executive_summary(self):
        """Generate a comprehensive executive summary of insights"""
        print("EXECUTIVE BUSINESS INTELLIGENCE SUMMARY")
        print("=" * 60)

        # Get key metrics
        total_products_query = f"SELECT COUNT(*) as total FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`"
        total_products = bpd.read_gbq(total_products_query).to_pandas().iloc[0]['total']

        categories_query = f"SELECT COUNT(DISTINCT category) as categories FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`"
        total_categories = bpd.read_gbq(categories_query).to_pandas().iloc[0]['categories']

        brands_query = f"SELECT COUNT(DISTINCT brand) as brands FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`"
        total_brands = bpd.read_gbq(brands_query).to_pandas().iloc[0]['brands']

        print(f"PORTFOLIO OVERVIEW:")
        print(f"Total Products: {total_products:,}")
        print(f"Categories: {total_categories}")
        print(f"Brands: {total_brands}")

        # Analyze cross-selling opportunities
        cross_sell = self.analyze_cross_selling_opportunities()
        print(f"\nTOP CROSS-SELLING OPPORTUNITIES:")
        for i, (_, row) in enumerate(cross_sell.head(3).iterrows(), 1):
            print(f"   {i}. {row['primary_category']} ↔ {row['cross_sell_category']}")
            print(f"      Semantic Affinity: {row['semantic_affinity']:.3f} | Bundle Value: ${row['potential_bundle_value']:.2f}")

        # Analyze pricing opportunities
        pricing = self.identify_pricing_opportunities()
        underpriced = pricing[pricing['pricing_recommendation'].str.contains('Under-priced')]
        overpriced = pricing[pricing['pricing_recommendation'].str.contains('Over-priced')]

        print(f"\nPRICING INSIGHTS:")
        print(f" Under-priced Products: {len(underpriced)}")
        print(f" Over-priced Products: {len(overpriced)}")
        if len(underpriced) > 0:
            avg_increase_opportunity = underpriced['price_gap_percent'].mean()
            print(f"   • Avg. Price Increase Opportunity: {avg_increase_opportunity:.1f}%")

        # Brand positioning insights
        brand_pos = self.analyze_brand_positioning()
        premium_brands = len(brand_pos[brand_pos['price_positioning'] == 'Premium Brand'])
        value_brands = len(brand_pos[brand_pos['price_positioning'] == 'Value Brand'])

        print(f"\nBRAND POSITIONING:")
        print(f" Premium Brands: {premium_brands}")
        print(f" Value Brands: {value_brands}")
        print(f" Most Competitive Space: {brand_pos.iloc[0]['brand']} (Score: {brand_pos.iloc[0]['market_similarity_score']})")

        return {
            'total_products': total_products,
            'cross_sell_opportunities': len(cross_sell),
            'pricing_opportunities': len(underpriced) + len(overpriced),
            'brand_insights': len(brand_pos)
        }

# Advanced Search & Discovery Features

In [22]:
class AdvancedSearchDiscovery:
    """Next-generation search and discovery features"""

    def __init__(self, project_id, dataset_id):
        self.project_id = project_id
        self.dataset_id = dataset_id

    def semantic_trend_analysis(self, category=None):
        """Analyze trends in product semantics within categories"""
        category_filter = f"AND t1.category = '{category}'" if category else ""

        query = f"""
        WITH category_avg_similarity AS (
            SELECT
                p1.category,
                AVG((1 - COSINE_DISTANCE(p1.embedding, p2.embedding))) as avg_intra_category_similarity
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
              ON p1.category = p2.category AND p1.id != p2.id
            GROUP BY p1.category
        ),
        brand_metrics AS (
            SELECT
                p.category,
                p.brand,
                COUNT(*) as product_count,
                AVG(p.retail_price) as avg_price
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
            WHERE TRUE {category_filter}
            GROUP BY p.category, p.brand
            HAVING COUNT(*) >= 3
        )
        SELECT
            bm.category,
            bm.brand,
            bm.product_count,
            ROUND(bm.avg_price, 2) as avg_price,
            ROUND(cas.avg_intra_category_similarity, 3) as semantic_coherence,
            CASE
                WHEN cas.avg_intra_category_similarity > 0.6 THEN 'Highly Focused Brand'
                WHEN cas.avg_intra_category_similarity > 0.4 THEN 'Moderately Focused'
                ELSE 'Diverse Portfolio'
            END as brand_focus_strategy
        FROM brand_metrics bm
        JOIN category_avg_similarity cas ON bm.category = cas.category
        ORDER BY semantic_coherence DESC, product_count DESC
        LIMIT 25
        """

        return bpd.read_gbq(query).to_pandas()

    def find_market_gaps(self):
        """Identify potential market gaps using semantic analysis"""
        query = f"""
        WITH category_embeddings AS (
            SELECT
                category,
                -- Calculate category centroid (average embedding) by unnesting
                ARRAY(
                    SELECT AVG(dim_value)
                    FROM (
                        SELECT embedding, GENERATE_ARRAY(0, ARRAY_LENGTH(embedding)-1) as positions
                        FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
                    ),
                    UNNEST(positions) as pos WITH OFFSET
                    CROSS JOIN UNNEST([embedding[SAFE_OFFSET(pos)]]) as dim_value
                    GROUP BY pos
                    ORDER BY pos
                ) as avg_embedding,
                COUNT(*) as product_count,
                AVG(retail_price) as avg_price
            FROM
                `{self.project_id}.{self.dataset_id}.product_embeddings_clean`,
                UNNEST(embedding) as value WITH OFFSET index
            GROUP BY category
            HAVING COUNT(*) >= 10
        ),
        category_distances AS (
            SELECT
                c1.category as category1,
                c2.category as category2,
                -- Use COSINE_DISTANCE for semantic similarity between centroids
                ROUND((1 - COSINE_DISTANCE(c1.avg_embedding, c2.avg_embedding)), 3) as semantic_similarity,
                ABS(c1.avg_price - c2.avg_price) as price_distance,
                c1.product_count as cat1_products,
                c2.product_count as cat2_products
            FROM category_embeddings c1
            CROSS JOIN category_embeddings c2
            WHERE c1.category != c2.category
        )
        SELECT
            category1,
            category2,
            -- Rename semantic_similarity to semantic_gap for consistency with original output
            semantic_similarity as semantic_gap,
            ROUND(price_distance, 2) as price_gap,
            cat1_products + cat2_products as total_products,
            CASE
                WHEN semantic_similarity < 0.35 AND price_distance > 30 THEN 'High Potential Gap' -- Lower similarity indicates a gap
                WHEN semantic_similarity < 0.5 AND price_distance > 20 THEN 'Moderate Gap'
                ELSE 'Low Priority Gap'
            END as market_opportunity
        FROM category_distances
        WHERE semantic_similarity BETWEEN 0.05 AND 0.6  -- Adjusted range for similarity (lower is gap)
        ORDER BY semantic_similarity ASC, price_distance DESC -- Order by lowest similarity first for gaps
        LIMIT 15
        """

        return bpd.read_gbq(query).to_pandas()

    def intelligent_bundle_suggestions(self, max_bundle_price=200):
        """Generate intelligent product bundles based on semantic similarity and business rules"""
        query = f"""
        WITH bundle_candidates AS (
            SELECT
                p1.id as product1_id,
                p1.name as product1_name,
                p1.category as product1_category,
                p1.retail_price as product1_price,
                p2.id as product2_id,
                p2.name as product2_name,
                p2.category as product2_category,
                p2.retail_price as product2_price,
                (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) as similarity,
                p1.retail_price + p2.retail_price as bundle_price
            FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p1
            CROSS JOIN `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p2
            WHERE p1.id < p2.id  -- Avoid duplicates
              AND p1.category != p2.category  -- Cross-category bundles
              AND (1 - COSINE_DISTANCE(p1.embedding, p2.embedding)) > 0.4
              AND p1.retail_price + p2.retail_price <= {max_bundle_price}
        )
        SELECT
            product1_name,
            product1_category,
            ROUND(product1_price, 2) as product1_price,
            product2_name,
            product2_category,
            ROUND(product2_price, 2) as product2_price,
            ROUND(bundle_price, 2) as bundle_price,
            ROUND(similarity, 3) as compatibility_score,
            CASE
                WHEN similarity > 0.7 THEN 'Perfect Bundle'
                WHEN similarity > 0.6 THEN 'Great Bundle'
                WHEN similarity > 0.5 THEN 'Good Bundle'
                ELSE 'Moderate Bundle'
            END as bundle_quality,
            -- Suggested discount
            ROUND(bundle_price * 0.1, 2) as suggested_discount,
            ROUND(bundle_price * 0.9, 2) as discounted_price
        FROM bundle_candidates
        ORDER BY similarity DESC, bundle_price ASC
        LIMIT 20
        """

        return bpd.read_gbq(query).to_pandas()

    def personalized_discovery_engine(self, user_preferences):
      """Advanced personalized product discovery"""
      # This would typically use user behavior data, but we'll simulate with preferences
      category_pref = user_preferences.get('preferred_category', 'Jeans')
      price_range = user_preferences.get('price_range', [20, 80])

      # Calculate expanded price range for recommendations
      min_price = price_range[0] * 0.7
      max_price = price_range[1] * 1.3

      query = f"""
      WITH user_profile AS (
          SELECT
              -- Calculate centroid by averaging each dimension
              ARRAY(
                  SELECT AVG(dim_value)
                  FROM (
                      SELECT embedding, GENERATE_ARRAY(0, ARRAY_LENGTH(embedding)-1) as positions
                      FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean`
                      WHERE category = '{category_pref}' AND retail_price BETWEEN {price_range[0]} AND {price_range[1]}
                  ),
                  UNNEST(positions) as pos WITH OFFSET
                  CROSS JOIN UNNEST([embedding[SAFE_OFFSET(pos)]]) as dim_value
                  GROUP BY pos
                  ORDER BY pos
              ) as avg_embedding
      ),
      personalized_scores AS (
          SELECT
              p.id,
              p.name,
              p.category,
              p.brand,
              p.retail_price,
              -- Calculate personalized similarity
              (1 - ML.DISTANCE(p.embedding, up.avg_embedding, 'COSINE')) as personalization_score,

              -- Diversity bonus for exploration
              CASE WHEN p.category != '{category_pref}' THEN 0.1 ELSE 0 END as diversity_bonus

          FROM `{self.project_id}.{self.dataset_id}.product_embeddings_clean` p
          CROSS JOIN user_profile up
          WHERE p.retail_price BETWEEN {min_price} AND {max_price}  -- Expanded range
      )
      SELECT
          name,
          category,
          brand,
          ROUND(retail_price, 2) as price,
          ROUND(personalization_score + diversity_bonus, 3) as recommendation_score,
          CASE
              WHEN category = '{category_pref}' THEN 'Perfect Match'
              WHEN personalization_score > 0.5 THEN 'Discovery Pick'
              ELSE 'Exploration Option'
          END as recommendation_type
      FROM personalized_scores
      WHERE personalization_score IS NOT NULL
      ORDER BY recommendation_score DESC
      LIMIT 15
      """

      print(f"personalized_discovery_engine = {query}")
      return bpd.read_gbq(query).to_pandas()

# EXECUTION SECTION: Run All Analysis

In [19]:
def run_complete_analysis():
    """Execute the complete vector search analysis pipeline"""

    print("STARTING COMPREHENSIVE VECTOR SEARCH ANALYSIS")
    print("=" * 80)

    # Initialize all components
    explorer = SmartProductExplorer(PROJECT_ID, DATASET_ID)
    rec_engine = AdvancedRecommendationEngine(PROJECT_ID, DATASET_ID)
    viz_dashboard = InteractiveVisualizationDashboard(PROJECT_ID, DATASET_ID)
    bi_engine = BusinessIntelligenceEngine(PROJECT_ID, DATASET_ID)
    search_discovery = AdvancedSearchDiscovery(PROJECT_ID, DATASET_ID)

    # Load products and basic exploration
    print("\nLOADING PRODUCTS WITH INSIGHTS...")
    products = explorer.load_products_with_insights()

    # Interactive search example
    print("\nSMART SEARCH DEMO:")
    search_results = explorer.smart_search("jacket")
    print(search_results[['name', 'category', 'brand', 'price_tier', 'brand_tier']].head())

    print("\nCATEGORY INSIGHTS:")
    category_insights = explorer.get_category_insights()
    print(category_insights.head(10))

    # Recommendation analysis
    print("\nRECOMMENDATION ENGINE DEMO:")
    sample_product_id = 21018  # You can change this
    recommendations_comparison = rec_engine.comprehensive_recommendation_comparison(sample_product_id)

    # Visualizations
    print("\nCREATING INTERACTIVE VISUALIZATIONS...")
    try:
        similarity_heatmap = viz_dashboard.create_similarity_heatmap()
        category_dashboard = viz_dashboard.create_category_performance_dashboard()
        if recommendations_comparison:
            rec_quality_viz = viz_dashboard.visualize_recommendation_quality(sample_product_id, recommendations_comparison)
    except Exception as e:
        print(f"Visualization error (likely due to environment): {e}")

    # Business Intelligence Analysis
    print("\nBUSINESS INTELLIGENCE ANALYSIS...")
    summary_metrics = bi_engine.generate_executive_summary()

    print("\n" + "="*60)
    print("DETAILED ANALYSIS RESULTS")
    print("="*60)

    # Cross-selling opportunities
    print(f"\nCROSS-SELLING OPPORTUNITIES:")
    cross_sell_results = bi_engine.analyze_cross_selling_opportunities()
    print(cross_sell_results.head(10).to_string(index=False))

    # Pricing opportunities
    print(f"\nPRICING OPTIMIZATION OPPORTUNITIES:")
    pricing_results = bi_engine.identify_pricing_opportunities()
    print(pricing_results.head(10).to_string(index=False))

    # Product clustering insights
    print(f"\nPRODUCT CLUSTERING INSIGHTS:")
    clustering_results = bi_engine.generate_product_clustering_insights()
    print(clustering_results.head(8).to_string(index=False))

    # Brand positioning analysis
    print(f"\nBRAND POSITIONING ANALYSIS:")
    brand_results = bi_engine.analyze_brand_positioning()
    print(brand_results.head(8).to_string(index=False))

    # Advanced search & discovery
    print(f"\nADVANCED SEARCH & DISCOVERY INSIGHTS")
    print("="*50)

    # Semantic trend analysis
    print(f"\nSEMANTIC TREND ANALYSIS:")
    trend_results = search_discovery.semantic_trend_analysis()
    print(trend_results.head(8).to_string(index=False))

    # Market gaps analysis
    # print(f"\nMARKET GAP OPPORTUNITIES:")
    # gap_results = search_discovery.find_market_gaps()
    # print(gap_results.head(8).to_string(index=False))

    # Bundle suggestions
    print(f"\nINTELLIGENT BUNDLE SUGGESTIONS:")
    bundle_results = search_discovery.intelligent_bundle_suggestions()
    print(bundle_results.head(6).to_string(index=False))

    # Personalized discovery demo
    print(f"\nPERSONALIZED DISCOVERY DEMO:")
    user_prefs = {
        'preferred_category': 'Jeans',
        'price_range': [25, 75]
    }
    personalized_results = search_discovery.personalized_discovery_engine(user_prefs)
    print(f"For user preferring {user_prefs['preferred_category']} in ${user_prefs['price_range'][0]}-{user_prefs['price_range'][1]} range:")
    print(personalized_results.head(8).to_string(index=False))

    # Final summary
    print("\n" + "="*80)
    print("VECTOR SEARCH ANALYSIS COMPLETE!")
    print("="*80)

    print(f"""
      ANALYSIS SUMMARY:
      ├── Products Analyzed: {summary_metrics['total_products']:,}
      ├── Cross-Sell Opportunities Found: {summary_metrics['cross_sell_opportunities']}
      ├── Pricing Optimization Cases: {summary_metrics['pricing_opportunities']}
      └── Brand Positioning Insights: {summary_metrics['brand_insights']}
    """)

    print("Analysis complete! All functions ready for interactive use.")

    return {
        'explorer': explorer,
        'rec_engine': rec_engine,
        'viz_dashboard': viz_dashboard,
        'bi_engine': bi_engine,
        'search_discovery': search_discovery,
        'summary_metrics': summary_metrics
    }


# INTERACTIVE FUNCTIONS FOR JUPYTER NOTEBOOKS

In [20]:

def quick_product_search(search_term, max_results=10):
    """Quick product search function"""
    explorer = SmartProductExplorer(PROJECT_ID, DATASET_ID)
    return explorer.smart_search(search_term, max_results)

def get_recommendations(product_id, algorithm='hybrid', top_k=5):
    """Get recommendations for a specific product"""
    rec_engine = AdvancedRecommendationEngine(PROJECT_ID, DATASET_ID)

    if algorithm == 'semantic':
        return rec_engine.semantic_recommendations(product_id, top_k)
    elif algorithm == 'business':
        return rec_engine.business_smart_recommendations(product_id, top_k)
    else:  # hybrid
        return rec_engine.hybrid_recommendations(product_id, top_k)

def analyze_category_performance():
    """Quick category performance analysis"""
    bi_engine = BusinessIntelligenceEngine(PROJECT_ID, DATASET_ID)
    return bi_engine.analyze_cross_selling_opportunities()

def find_bundle_opportunities(max_price=200):
    """Find bundle opportunities"""
    search_discovery = AdvancedSearchDiscovery(PROJECT_ID, DATASET_ID)
    return search_discovery.intelligent_bundle_suggestions(max_price)

def get_pricing_insights():
    """Get pricing optimization insights"""
    bi_engine = BusinessIntelligenceEngine(PROJECT_ID, DATASET_ID)
    return bi_engine.identify_pricing_opportunities()

# MAIN EXECUTION

In [23]:
if __name__ == "__main__":
    # Run the complete analysis
    results = run_complete_analysis()

    # Store results in global variables for interactive use
    explorer = results['explorer']
    rec_engine = results['rec_engine']
    viz_dashboard = results['viz_dashboard']
    bi_engine = results['bi_engine']
    search_discovery = results['search_discovery']

    print("\nINTERACTIVE MODE READY!")
    print("Available functions:")
    print("quick_product_search('search_term')")
    print("get_recommendations(product_id, algorithm='hybrid')")
    print("analyze_category_performance()")
    print("find_bundle_opportunities(max_price=200)")
    print("get_pricing_insights()")
    print("\nAll class instances are available as global variables:")
    print("explorer, rec_engine, viz_dashboard, bi_engine, search_discovery")

STARTING COMPREHENSIVE VECTOR SEARCH ANALYSIS

LOADING PRODUCTS WITH INSIGHTS...
Loaded 29094 products with insights

SMART SEARCH DEMO:
Found 1917 products matching 'jacket'
                                                 name           category  \
20              KASPER Metallic 2PC Jacket/Dress Suit              Suits   
25  Voom by Joy Han Capelet Jacket - Pinstripe in ...  Blazers & Jackets   
34  Port Authority Ladies Long Textured Hooded Sof...  Outerwear & Coats   
57  Onyx-Arctic Shield-X-System Men's Arcticshield...  Outerwear & Coats   
65  Allegra K Ladies Long Sleeve Knit Flower Neck ...  Blazers & Jackets   

                          brand          price_tier      brand_tier  
20                       Kasper      Luxury ($100+)   Popular Brand  
25                         Voom  Premium ($50-$100)  Boutique Brand  
34               Port Authority  Premium ($50-$100)   Popular Brand  
57  Onyx-Arctic Shield-X-System  Premium ($50-$100)  Boutique Brand  
65                


BUSINESS INTELLIGENCE ANALYSIS...
EXECUTIVE BUSINESS INTELLIGENCE SUMMARY
PORTFOLIO OVERVIEW:
Total Products: 29,094
Categories: 26
Brands: 2756

TOP CROSS-SELLING OPPORTUNITIES:
   1. Socks & Hosiery ↔ Socks
      Semantic Affinity: 0.631 | Bundle Value: $37.18
   2. Socks ↔ Socks & Hosiery
      Semantic Affinity: 0.631 | Bundle Value: $37.18
   3. Pants & Capris ↔ Pants
      Semantic Affinity: 0.618 | Bundle Value: $114.52

PRICING INSIGHTS:
 Under-priced Products: 3
 Over-priced Products: 27
   • Avg. Price Increase Opportunity: 42.1%

BRAND POSITIONING:
 Premium Brands: 1
 Value Brands: 9
 Most Competitive Space: Slimme (Score: 0.59)

DETAILED ANALYSIS RESULTS

CROSS-SELLING OPPORTUNITIES:
 primary_category cross_sell_category  similar_product_pairs  semantic_affinity  avg_primary_price  avg_cross_sell_price  potential_bundle_value   opportunity_rating
  Socks & Hosiery               Socks                 600495              0.631              16.76                 20.42        